# DELE

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
from keras import models, utils, layers

## Part A > CNN

### Data Preparation

In [ ]:

(X_train, y_train), (X_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

X_train = X_train.astype("float32") / 255
X_test = X_test.astype("float32") / 255

X_train = np.expand_dims(X_train, -1)
X_test = np.expand_dims(X_test, -1)

y_train = tf.keras.utils.to_categorical(y_train, 10)
y_test = tf.keras.utils.to_categorical(y_test, 10)

In [ ]:
model = models.Sequential([
    layers.Input(shape=(28, 28, 1)),
    layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Flatten(),
    layers.Dropout(0.5),
    layers.Dense(10, activation="softmax"),
])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=10, batch_size=128, validation_split=0.1)


In [ ]:
def confusion_matrix(X_test, y_test, model):
    y_pred = model.predict(X_test).argmax(axis=1)
    y_true = y_test.argmax(axis=1)
    df = pd.DataFrame(
        data=np.zeros(shape=(10, 10)).astype(int),
        index=pd.Index(data=np.arange(10), name='Prediction'),
        columns=pd.Index(data=np.arange(10), name='Actual')
    )
    correct = 0
    wrong = 0
    for i in range(len(y_true)):
        df[y_pred[i]][y_true[i]] += 1
        if y_pred[i] == y_true[i]:
            correct += 1
        else:
            wrong += 1
    
    return df, correct / (correct + wrong) * 100.0

# print(confusion_matrix(X_test, y_test, model))

In [ ]:
model.evaluate(X_test, y_test)

## Part B > RNN

In [ ]:
(X_train_2, y_train_2), (X_test_2, y_test_2) = tf.keras.datasets.cifar10.load_data()

In [ ]:
X_train_2.astype(float)
X_test_2.astype(float)

y_train_2 = tf.keras.utils.to_categorical(y_train_2, 10)
y_test_2 = tf.keras.utils.to_categorical(y_test_2, 10)

model2 = models.Sequential([
    layers.Rescaling(1 / 255),
    layers.Input(shape=(32, 32, 3)),
    layers.Conv2D(32, kernel_size=(3, 3), activation="relu"),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Conv2D(64, kernel_size=(3, 3), activation="relu"),
    layers.MaxPooling2D(pool_size=(2, 2)),
    layers.Flatten(),
    layers.Dropout(0.5),
    layers.Dense(10, activation="softmax"),
])
model2.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model2.fit(X_train_2, y_train_2, epochs=3, batch_size=10, validation_split=0.1)


In [ ]:
print(confusion_matrix(X_test_2, y_test_2, model2)[0])

In [ ]:
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from typing import List, Tuple
from sklearn.model_selection import RandomizedSearchCV

def create_model(hidden_layers: List[List[int]], optimiser: str, activation: str, input_shape: Tuple[int]):
    model = models.Sequential()
    model.add(layers.Input(shape=input_shape))
    for hidden_layer in hidden_layers:
        model.add(layers.Dense(hidden_layer, activation=activation))
        model.add(layers.Conv2D(32, kernel_size=3))
    model.add(layers.Flatten())
    model.add(layers.Dense(10, activation='softmax'))
    model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

clf1 = KerasClassifier(build_fn=create_model, verbose=0)

params = {
    'optimiser': ['adam', 'rmsprop'],
    'hidden_layers': [
        [32, 64],
        [32, 64, 128],
    ],
    'epochs': [1],
    'batch_size': [100],
    'activation': ['relu'],
    'input_shape': [(28, 28, 1)]
}

gs = RandomizedSearchCV(estimator=clf1, param_distributions=params, n_iter=1)
gs_result = gs.fit(X_train, y_train)

In [ ]:
pd.DataFrame(gs_result.cv_results_)

In [ ]:
# import pickle

# with open('./models/model1.p', 'rb') as f:
#     print(pickle.load(f))